In [1]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("NaiveBayes_Example").getOrCreate()

workspace_default_storage_account = "projectgstoragedfb938a3e"
workspace_default_container = "azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f"
workspace_wasbs_base_url = f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"


StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 44, 6, Finished, Available, Finished)

In [4]:
from pyspark.sql.functions import lit

cancer_path = f"{workspace_wasbs_base_url}cancer_subreddit_sentiment.parquet"
cancer_df = spark.read.parquet(cancer_path)

not_cancer_path = f"{workspace_wasbs_base_url}not_cancer_subreddit_sentiment.parquet"
not_cancer = spark.read.parquet(not_cancer_path)

cancer_df = cancer_df.withColumn("source", lit("cancer"))

not_cancer = not_cancer.withColumn("source", lit("non_cancer"))

combined_df = cancer_df.union(not_cancer)
combined_df = combined_df.select('text', 'source')

combined_df.show()



StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 44, 9, Finished, Available, Finished)

+--------------------+------+
|                text|source|
+--------------------+------+
|Check out Northsi...|cancer|
|I had something s...|cancer|
|That's an insulti...|cancer|
|Yeah sorry, it wa...|cancer|
|I see my colorect...|cancer|
|The couple of ran...|cancer|
|I’ve encountered ...|cancer|
|I 100% agree with...|cancer|
|You should not ha...|cancer|
|**Your post has b...|cancer|
|Completely agree ...|cancer|
|Butt's have oil s...|cancer|
|I just found I ha...|cancer|
|in the mid 2000s ...|cancer|
|If you know all y...|cancer|
|Ultimately, us nu...|cancer|
|I had one the siz...|cancer|
|yeah I see that n...|cancer|
|This has been a r...|cancer|
|Day to day is dif...|cancer|
+--------------------+------+
only showing top 20 rows



In [5]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Tokenizer and stop words removal
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

# Vectorization and TF-IDF
count_vectorizer = CountVectorizer(inputCol="filtered_tokens", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")

# Label encoding
indexer = StringIndexer(inputCol="source", outputCol="label")

# Build pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, indexer])

# Fit and transform
processed_df = pipeline.fit(combined_df).transform(combined_df)

# Build pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, indexer])

# Fit and transform
processed_df = pipeline.fit(combined_df).transform(combined_df)

# Split the data 80-20
train_data, test_data = processed_df.randomSplit([0.8, 0.2], seed=42)

# Train Naive Bayes model
nb = NaiveBayes(featuresCol="features", labelCol="label", modelType="multinomial")
nb_model = nb.fit(train_data)

# Predict
predictions = nb_model.transform(test_data)

# Evaluate
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

evaluator = MulticlassClassificationEvaluator()
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")


StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 44, 10, Submitted, Running, Running)

In [10]:
# Weighted Precision (Multiclass)
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)
print(f"Test Precision: {precision:.2f}")

# Weighted Recall (Multiclass)
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)
print(f"Test Recall: {recall:.2f}")

# F1-Score (Multiclass)
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f"Test F1-Score: {f1_score:.2f}")

# AUC-ROC (For Binary Classification, works for binary classification)
evaluator_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
auc_roc = evaluator_auc.evaluate(predictions)
print(f"Test AUC-ROC: {auc_roc:.2f}")

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 37, 15, Finished, Available, Finished)

Test Precision: 0.55
Test Recall: 0.31
Test F1-Score: 0.39
Test AUC-ROC: 0.47


In [3]:


# Split the data 80-20
train_data, test_data = processed_df.randomSplit([0.8, 0.2], seed=42)

# Train Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)

# Predict
predictions = lr_model.transform(test_data)

# Evaluate
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col

# Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.2f}")

# Precision
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="precision")
precision = evaluator_precision.evaluate(predictions)
print(f"Test Precision: {precision:.2f}")

# Recall
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="recall")
recall = evaluator_recall.evaluate(predictions)
print(f"Test Recall: {recall:.2f}")

# F1-Score
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f"Test F1-Score: {f1_score:.2f}")

# AUC-ROC (for binary classification)
evaluator_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
auc_roc = evaluator_auc.evaluate(predictions)
print(f"Test AUC-ROC: {auc_roc:.2f}")

# Confusion Matrix
prediction_and_labels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(prediction_and_labels)
print("Confusion Matrix:")
print(metrics.confusionMatrix().toArray())



StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 37, 8, Finished, Available, Finished)

Test Accuracy: 0.58


In [ ]:
# Weighted Precision (Multiclass)
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)
print(f"Test Precision: {precision:.2f}")

# Weighted Recall (Multiclass)
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)
print(f"Test Recall: {recall:.2f}")

# F1-Score (Multiclass)
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f"Test F1-Score: {f1_score:.2f}")

# AUC-ROC (For Binary Classification, works for binary classification)
evaluator_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
auc_roc = evaluator_auc.evaluate(predictions)
print(f"Test AUC-ROC: {auc_roc:.2f}")